In [1]:
import argparse
import os
import pickle

import torch as T
import yaml
from torch.optim import Adam
from tqdm import tqdm

import sys
sys.path.append('/home/volpepe/FSOD_CenterNet/src')
from data_pipeline import DatasetsGenerator
from model import Model


def load_settings(settings_path: str):
    with open(settings_path, 'r') as f:
        return yaml.safe_load(f)
    
settings = '../settings/model_testing_debug.yaml'
    
config = load_settings(settings)

if os.path.exists(config['training']['save_base_weights_dir']) \
                and config["training"]["train_base"] \
                and config["training"]["no_overwrite"]:
    raise ValueError("Cannot overwrite weights")

os.makedirs(config['training']['save_training_info_dir'], exist_ok=True)

debug_mode = config['debug']['debug_mode_active']
device = config['device']

K = config['data']['K']
val_K = config['data']['val_K']
test_K = config['data']['test_K']
n_repeats_novel_train = config['training']['repeat_novel_training']

if isinstance(K, int): K = [K]
if isinstance(val_K, int): val_K = [val_K]
if isinstance(test_K, int): test_K = [test_K]

# Dataset generator. Only one of these has to be instantiated. It always returns
dataset_gen = DatasetsGenerator(config)

if config['training']['train_base']:

    # Instantiate the model (only the base part)
    model = Model(config, n_base_classes=len(dataset_gen.train_base.cats))
    model = model.to(device)

    # Use the dataset generator to generate the base set
    dataset_base_train, dataset_base_val, dataset_base_test = dataset_gen.get_base_sets_dataloaders(
        config['training']['batch_size'], config['training']['num_workers'],
        config['training']['pin_memory'], config['training']['drop_last'], 
        shuffle=True
    )
    

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=3.77s)
creating index...
index created!
Skipping instantating novel head


In [2]:
model.load_state_dict(T.load('../../data/weights/best_model.pt'))

<All keys matched successfully>

In [3]:
for i, (input_image, labels, n_detections, _) in enumerate(dataset_base_train):
    out = model(input_image.to(device))
    break

In [5]:
out_reg, out_head_base, _ = out

In [17]:
regressor_label, heatmap_base, _ = labels

In [20]:
out_reg[T.where(regressor_label != 0)]

tensor([2.0334, 2.4795, 0.4907, 0.5039, 2.4553, 1.9505, 2.9361, 2.3897, 0.4902,
        0.4908, 0.5038, 0.5039], device='cuda:0', grad_fn=<IndexBackward0>)

In [21]:
regressor_label[T.where(regressor_label != 0)]

tensor([ 3.5229,  2.0454,  0.6521,  0.4062, 11.8698,  8.8156,  7.7714, 15.4122,
         0.1491,  0.9951,  0.7622,  0.3418])

In [31]:
T.where(heatmap_base==1)

(tensor([0, 1, 1]),
 tensor([ 88, 145, 145]),
 tensor([3, 2, 3]),
 tensor([3, 4, 6]))

In [30]:
out_head_base

tensor([[[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          ...,
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          ...,
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],

         [[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000,  ..., 0

In [44]:
x = model.state_dict()

In [52]:
x['head_base_heatmap.conv1.weight']

tensor([[[[-9.8056e-41,  6.4492e-41,  8.6780e-41],
          [ 5.0696e-40, -1.6632e-40,  2.9484e-40],
          [-4.6411e-40,  2.4445e-40,  4.3672e-40]],

         [[-2.1020e-40, -2.0201e-40, -2.7835e-40],
          [ 4.1332e-40, -3.7839e-40,  4.5052e-40],
          [-3.0858e-40, -7.8785e-41,  2.9972e-40]],

         [[-4.8492e-40, -4.8449e-40, -6.0444e-40],
          [ 5.3649e-41, -1.8489e-40,  1.8786e-41],
          [-5.1730e-40, -4.9594e-40, -5.3414e-40]],

         ...,

         [[-1.2725e-40,  1.5731e-40,  3.4241e-40],
          [-4.2024e-40, -3.0144e-40, -4.5437e-40],
          [ 4.2116e-40, -4.4183e-40, -1.5510e-40]],

         [[-3.7219e-26, -4.4999e-26, -3.7044e-26],
          [-4.1545e-26, -4.4576e-26, -3.9690e-26],
          [-3.4183e-26, -4.5153e-26, -3.7952e-26]],

         [[ 1.5845e-40, -8.6963e-41, -4.1071e-41],
          [ 3.6266e-41, -8.6768e-42, -1.4405e-41],
          [ 2.1300e-41,  7.4988e-41, -1.1044e-41]]],


        [[[ 6.2159e-40,  5.8929e-40, -7.2342e-41],
  